In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

class TreePartitionMethod:
    def __init__(self, data, num_intervals):
        self.data = data
        self.num_intervals = num_intervals
        self.universe = (min(data), max(data))
        self.intervals = self._initial_partition()
        self.frequencies = self._calculate_frequencies()
        self.avg_frequency = np.mean(self.frequencies)
        self.optimized_intervals = self._optimize_intervals()

    def _initial_partition(self):
        min_val, max_val = self.universe
        interval_length = (max_val - min_val) / self.num_intervals
        intervals = [(min_val + i * interval_length, min_val + (i + 1) * interval_length) for i in range(self.num_intervals)]
        return intervals

    def _calculate_frequencies(self):
        frequencies = []
        for low, high in self.intervals:
            count = sum(low <= x < high for x in self.data)
            frequencies.append(count)
        return frequencies

    def _optimize_intervals(self):
        optimized_intervals = self.intervals[:]
        frequencies = self.frequencies[:]
        avg_frequency = self.avg_frequency

        while True:
            new_intervals = []
            new_frequencies = []

            for i, (low, high) in enumerate(optimized_intervals):
                if frequencies[i] > avg_frequency:
                    mid = (low + high) / 2
                    new_intervals.append((low, mid))
                    new_intervals.append((mid, high))
                    new_frequencies.append(sum(low <= x < mid for x in self.data))
                    new_frequencies.append(sum(mid <= x < high for x in self.data))
                else:
                    new_intervals.append((low, high))
                    new_frequencies.append(frequencies[i])

            if new_intervals == optimized_intervals:
                break

            optimized_intervals = new_intervals
            frequencies = new_frequencies

        return optimized_intervals

    def get_intervals(self):
        return self.optimized_intervals

class FuzzyTimeSeriesMarkovChain:
    def __init__(self, data, intervals):
        self.data = data
        self.intervals = intervals
        self.fuzzy_sets = self._create_fuzzy_sets()
        self.transition_matrix = self._create_transition_matrix()

    def _create_fuzzy_sets(self):
        fuzzy_sets = []
        for value in self.data:
            fuzzy_set = [1 if low <= value < high else 0 for low, high in self.intervals]
            fuzzy_sets.append(fuzzy_set)
        return fuzzy_sets

    def _create_transition_matrix(self):
        num_intervals = len(self.intervals)
        transition_matrix = np.zeros((num_intervals, num_intervals))
        for i in range(len(self.fuzzy_sets) - 1):
            current_set = self.fuzzy_sets[i]
            next_set = self.fuzzy_sets[i + 1]
            current_index = current_set.index(1)
            next_index = next_set.index(1)
            transition_matrix[current_index][next_index] += 1

        # Normalize the transition matrix
        for i in range(len(transition_matrix)):
            total = sum(transition_matrix[i])
            if total != 0:
                transition_matrix[i] = transition_matrix[i] / total
        return transition_matrix

    def predict(self, steps):
        predictions = []
        current_state = self.fuzzy_sets[-1].index(1)
        for _ in range(steps):
            next_state = np.argmax(self.transition_matrix[current_state])
            predictions.append((self.intervals[next_state][0] + self.intervals[next_state][1]) / 2)  # Predict the midpoint of the interval
            current_state = next_state
        return predictions

    def evaluate(self, actual_data):
        predictions = self.predict(len(actual_data))
        mse = mean_squared_error(actual_data, predictions)
        rmse = np.sqrt(mse)
        mape = mean_absolute_percentage_error(actual_data, predictions)
        thiels_u = self._thiels_u(actual_data, predictions)
        return mse, rmse, mape, thiels_u

    def _thiels_u(self, actual, predicted):
        actual = np.array(actual)
        predicted = np.array(predicted)
        numerator = np.sqrt(np.mean((actual - predicted)**2))
        denominator = np.sqrt(np.mean(actual**2)) + np.sqrt(np.mean(predicted**2))
        return numerator / denominator

# Sample Data
data = [112, 115, 120, 130, 140, 150, 160, 170, 180, 190, 200]

# Tree Partition Method
tpm = TreePartitionMethod(data, num_intervals=5)

# Get optimized intervals
optimized_intervals = tpm.get_intervals()
print("Optimized Intervals:", optimized_intervals)

# Fuzzy Time Series Markov Chain Model
fts_mc = FuzzyTimeSeriesMarkovChain(data, optimized_intervals)

# Predict future values
predictions = fts_mc.predict(steps=5)
print("Predictions:", predictions)

# Evaluate the model (using a hypothetical future dataset)
# Replace `actual_future_data` with the actual future data you want to evaluate against.
actual_future_data = [210, 220, 230, 240, 250]
rmse, mape, thiels_u = fts_mc.evaluate(actual_future_data)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("Thiel's U:", thiels_u)
